In [741]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle

from datetime import timedelta

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [742]:
dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

# print(os.environ["db_name"])

# conn = pymysql.connect(
#   host = os.environ['db_host'],
#   port = int(os.environ['db_port']),
#   user = os.environ['db_user'],
#   password = os.environ['db_password'],
#   db = os.environ['db_name']
#   # charset= os.environ['db_charset']
# )

False

In [743]:
# curs = conn.cursor()

# sql = "SELECT cpg, DATE_FORMAT(save_time_id, '%Y-%m-%d'), HOUR(save_time) FROM gsmon_solar_data WHERE rtu_id='24' AND lerr != '57'"
# curs.execute(sql)

# result = curs.fetchall()

# print(result)

In [744]:
# with open('plant_data', 'wb') as file :
#   pickle.dump(result, file)

In [745]:
# with open( "plant_data", "rb" ) as file:
#     data = pickle.load(file)

In [746]:
plant_data = pd.read_pickle('plant_data')

data = pd.DataFrame(plant_data, columns=['누적발전량', '날짜', '시간'])

data.tail()

,누적발전량,날짜,시간
920524,238822000,2023-06-28,19
920525,238822000,2023-06-28,19
920526,238822000,2023-06-28,19
920527,238822000,2023-06-28,19
920528,238822000,2023-06-28,19


In [747]:
# 날짜 분리하기
data['날짜'] = pd.to_datetime(data['날짜'])

data['년'] = data['날짜'].dt.year
data['월'] = data['날짜'].dt.month
data['일'] = data['날짜'].dt.day

data = data[['날짜', '년','월','일','시간','누적발전량']]

year = [2020, 2023]

filter_y = data['년'].isin(year)
pg_data = data.drop(index=data[filter_y].index, axis=0)

pg_data.tail()

,날짜,년,월,일,시간,누적발전량
796473,2022-12-31,2022,12,31,17,204302000
796474,2022-12-31,2022,12,31,17,204302000
796475,2022-12-31,2022,12,31,17,204302000
796476,2022-12-31,2022,12,31,17,204302000
796477,2022-12-31,2022,12,31,17,204302000


In [748]:
# 2021년 1월 1일의 전일 데이터 처리를 위한 2020년 12월 31일 마지막 row 가져오기

filter_l = data['년']==2020

last_data_2020 = data[filter_l]

last_data = (last_data_2020.iloc[-1, :]).to_frame()

last_data= last_data.transpose()

print(last_data.shape)
print(pg_data.shape)

pg_data = pd.concat([last_data, pg_data], axis=0)

pg_data.head()

(1, 6)
(532266, 6)


,날짜,년,월,일,시간,누적발전량
264211,2020-12-31,2020,12,31,17,72059000
264212,2021-01-01,2021,1,1,7,72059000
264213,2021-01-01,2021,1,1,7,72059000
264214,2021-01-01,2021,1,1,7,72059000
264215,2021-01-01,2021,1,1,7,72059000


In [749]:
# 전일 최대 누적 발전량 column 추가

for i in range(1, len(pg_data)) :
  yesterday_date = pg_data.iloc[i]['날짜']-timedelta(days=1)
  # print(yesterday_date)
  filter_d = pg_data['날짜'] == yesterday_date.strftime('%Y-%m-%d')
  pg_data.iloc[i]['전일최대누적발전량'] = max(pg_data[filter_d]['누적발전량'])

# pg_data = pg_data.reset_index(drop=True)

# pg_data.loc[0]['누적발전량']

# for문 적용 테스트
# yesterday_date = pg_data.iloc[1]['날짜']-timedelta(days=1)
# filter_d = pg_data['날짜'] == yesterday_date.strftime('%Y-%m-%d')
# print(max(pg_data[filter_d]['누적발전량']))
# print(yesterday_date.strftime('%Y-%m-%d'))

72213000
2021-01-01


In [750]:
# pg_data = pg_data.iloc[1:, :]
print(pg_data.head())

# with open('plant_data_addy', 'wb') as file :
#   pickle.dump(pg_data, file)

               날짜     년   월   일  시간     누적발전량
264211 2020-12-31  2020  12  31  17  72059000
264212 2021-01-01  2021   1   1   7  72059000
264213 2021-01-01  2021   1   1   7  72059000
264214 2021-01-01  2021   1   1   7  72059000
264215 2021-01-01  2021   1   1   7  72059000
